# Heart Disease Risk Factor Analysis
## Using classification to predict heart disease diagnosis of patients in Cleveland 
### DSCI100 100 Group 3 - Chinchin Jin, Kadek Ariska Gangga Dewi, Mark Hao Jun Pan, Endo Cui

## Introduction 

Heart disease describes a variety of conditions that affect the heart which include, but are not limited to, blood vessel diseases, irregular heartbeats, congenital heart defects, heart muscle diseases, and heart valve diseases. [1] The severity and diversity of heart disease makes it a major public health concern: according to the National Center for Health Statistics, heart disease stands as the leading cause of death for both men and women in the United States. [2] This statistic underscores the urgency of understanding, monitoring, and predicting the presence of heart disease in individuals.

Given its prevalence and the potentially life-threatening consequences, effective observation and prediction of heart disease become important. Early detection and intervention can significantly improve outcomes, prevent complications, and ultimately save lives. Additionally, focused analysis on predictors such as age,heart ratee, and cholesterol will help build more timely and targeted interventions to mitigate the impact of heart disease on individuals and the broader population. 

This project aims to use several variables to predict based on a patient's condition if heart disease is present. Thus, the question we intend on testing is as follows: considering age, maximum heart rate achieved, and chest pain type as predictors, are we able to predict the presence of heart disease in a patient?

We will be using the Heart Disease multivariate dataset downloaded from the UC Irvine Machine Learning Repository. Specifically, we will be using the Cleveland database subset from the Heart Disease database, and therefore we will predict solely on Cleveland residents. Within the dataset, there are 14 variables, and they are listed below with their definitions. 

1) **age**: Age of patient (years)
2) **sex**: Sex of patient
3) **cp**: Chest pain type (1 = typical angina; 2 = atypical angina; 3 = non-anginal pain; 4 = asymptomatic)
4) **trestbps**: Resting blood pressure (mmHg)
5) **chol**: Serum cholesterol (mg/dl)
6) **fbs**: Fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
7) **restecg**: Resting electrocardiographic results (1 = normal; 2 = having ST-T wave abnormality; 3 = showing probable or definite left ventricular hypertrophy by Estes' critera)
8) **thalach**: Maximum heart rate reached
9) **exang**: Exercise-induced angina (1 = true, 0 = false)
10) **oldpeak**: ST depression induced by exercise relative to rest
11) **slope**: Slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downslope)
12) **thal**: 3 = normal; 6 = fixed defect; 7 = reversible defect
13) **ca**: Number of major vessels (0-3) coloured by fluoroscopy
14) **num**: Diagnosis of heart disease (0 = negative; > 0 = positive)

## Methods & Results
Before we begin anything else, let us first load the necessary functions and the dataset. We will clean the dataest by dropping rows with empty values and also replacing the `num` variable with `heart_disease` and its corresponding variables, where a positive heart disease diagnosis is `True` and a negative diagnosis is `False`.  


In [5]:
import random
import copy
import altair as alt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (GridSearchCV,RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.compose import make_column_selector
from sklearn.metrics import recall_score, precision_score


In [6]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [7]:
from ucimlrepo import fetch_ucirepo 

heart_disease = fetch_ucirepo(id=45) 

X = heart_disease.data.features 
y = heart_disease.data.targets 

In [8]:
data = pd.read_csv("https://archive.ics.uci.edu/static/public/45/data.csv")
data['heart_disease'] = data['num'] > 0
data1 = data[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak",	"slope", "ca", "thal", "heart_disease"]].dropna()
data1

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heart_disease
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,False
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,True
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,True
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,False
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,4,140,241,0,0,123,1,0.2,2,0.0,7.0,True
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,True
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,True
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,True


**Table 1 - Cleaned dataframe**

For this question, we trained a K-nearest neighbors model to several selected predictor variables to predict the presence of heart disease in a patient. Thereafter, we evaluated the model based on the metrics accuracy and recall. In this project, while accuracy of the model is still a very important factor in evaluating the model, recall would take a greater precedence based on the nature of the model. Accuracy in our model would measure the total correct predicted diagnoses, whereas recall would measure the total number of correct positive diagnoses. False negatives in the context of heart disease diagnosis could potentially be fatal, and therefore it would be vital for the model to maximize recall.

Our initial proposal selected variables based on their observed trends and correlations as observed by the project members. This was a very subjective method of analyzing for patterns and meant some inconspicuous correlations not immediately discernible from the graphics generated were overlooked. Instead, the project used forward selection for the selection of appropriate predictor variables. The forward selection taught in the textbook is designed for computing maximal accuracy: this was tweaked to compute for combinations of maximal recall.


In [19]:
data_train, data_test = train_test_split(
    data1, train_size = 0.75, random_state = 400
)

First we randomly split the data into 75% training data and 25% testing data. We will run forward selection on the training data, as running forward selection on the entire dataset will result in biased results as the model will have 'peaked' at the testing data. We have set `random_state` to 400 to make it reproducible. 

In [20]:
names = list(data_train.drop(
    columns=["heart_disease"]
).columns.values)

accuracy_dict = {"size": [], "selected_predictors": [], "accuracy": []}

n_total = len(names)

selected = []

param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 100, 5),
}
preprocessor = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include="number"))
)
tune_pipe = make_pipeline(preprocessor, KNeighborsClassifier())
tune_grid = GridSearchCV(
    estimator=tune_pipe,
    param_grid=param_grid,
    cv=10,
    n_jobs=-1
)

for i in range(1, n_total + 1):
    accs = np.zeros(len(names))

    for j in range(len(names)):

        X = data_train[selected + [names[j]]]
        y = data_train["heart_disease"]

        tune_grid.fit(X, y)
        accuracies_grid = pd.DataFrame(tune_grid.cv_results_)

        accs[j] = accuracies_grid["mean_test_score"].max()

    best_set = selected + [names[accs.argmax()]]

    accuracy_dict["size"].append(i)
    accuracy_dict["selected_predictors"].append(", ".join(best_set))
    accuracy_dict["accuracy"].append(accs.max())

    selected = best_set
    del names[accs.argmax()]

recall = pd.DataFrame(accuracy_dict)
recall.sort_values(by="accuracy", ascending=False).rename(columns={"accuracy":"recall"})

,size,selected_predictors,recall
6,7,"thal, ca, slope, exang, cp, chol, trestbps",0.860474
7,8,"thal, ca, slope, exang, cp, chol, trestbps, re...",0.860277
5,6,"thal, ca, slope, exang, cp, chol",0.855929
10,11,"thal, ca, slope, exang, cp, chol, trestbps, re...",0.851383
8,9,"thal, ca, slope, exang, cp, chol, trestbps, re...",0.851186
9,10,"thal, ca, slope, exang, cp, chol, trestbps, re...",0.851186
11,12,"thal, ca, slope, exang, cp, chol, trestbps, re...",0.847036
4,5,"thal, ca, slope, exang, cp",0.846838
12,13,"thal, ca, slope, exang, cp, chol, trestbps, re...",0.846838
3,4,"thal, ca, slope, exang",0.837945


**Table 2 - Maximizing forward selection with recall**

Given the information from Table 2 above, we are given combinations of predictor variables to maximize recall. Running with respect to the training data results in an optimized variable combination of thal, ca, slope, exang, cp, chol, trestbps with a reported recall of ~0.86. Following combinations have similar but lower recall values. Therefore, because we are seeking the greatest recall, we will use these 7 variables. The variables are as defined: 

- **thal**: 3 = normal; 6 = fixed defect; 7 = reversible defect
- **ca**: Number of major vessels (0-3) coloured by fluoroscopy
- **slope**: Slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downslope)
- **exang**: Exercise-induced angina (1 = true, 0 = false)
- **cp**: Chest pain type (1 = typical angina; 2 = atypical angina; 3 = non-anginal pain; 4 = asymptomatic)
- **chol**: Serum cholesterol (mg/dl)
- **trestbps**: Resting blood pressure (mmHg)

Now that we have selected the predictor variables to use, we will proceed to making the classification model. Firstly, let us scale the numerical variables from the list and pass through all other variables: we know which variables are numerical from the repository where the dataset was obtained from. 

In [21]:
data_preprocessor = make_column_transformer(
    (StandardScaler(), ["ca", "chol", "trestbps"]),
    remainder = "passthrough",
    verbose_feature_names_out = True
)
data_preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('standardscaler', StandardScaler(),
                                 ['ca', 'chol', 'trestbps'])])

Now let us train and tune the model. We will run a *k*-nearest neighbors algorithm with a preliminary *k* = 2 and generate a confusion matrix. 

In [24]:
knn = KNeighborsClassifier(n_neighbors = 2)

X_train = data_train[["thal", "ca", "slope", "exang", "cp", "chol", "trestbps"]]
y_train = data_train["heart_disease"]

knn_pipeline = make_pipeline(data_preprocessor, knn)
knn_pipeline.fit(X_train, y_train)

data_test_predictions = data_test.assign(
    predict = knn_pipeline.predict(data_test[["thal", "ca", "slope", "exang", "cp", "chol", "trestbps"]])
)
data_test_predictions

X_test = data_test[["thal", "ca", "slope", "exang", "cp", "chol", "trestbps"]]
y_test = data_test["heart_disease"]

data_prediction_accuracy = knn_pipeline.score(X_test, y_test)
data_prediction_accuracy

data_mat = pd.crosstab(
    data_test_predictions["heart_disease"], 
    data_test_predictions["predict"]
)
data_mat

predict,False,True
heart_disease,,
False,39,4
True,11,21


**Confusion Matrix 1 - Model trial with *k* = 2**

We see that the accuracy of this trial *k* = 2 is approximately 0.80, and the recall is approximately 0.656. We can see from the confusion matrix generated above that the recall of *k* = 2 is not terrible, but leaves much to be improved. To find the optimal *k*-value, we will cross-validate the model with our validation set and find a *k* which maximizes recall and accuracy. We choose to run 10-fold validation, and we will generate a graph to compare recall and number of *k*. We set a random seed of 1234 to ensure it is reproducible. 

In [31]:
np.random.seed(1234)

data_pipe = make_pipeline(data_preprocessor, knn)
data_vfold_score = pd.DataFrame(
    cross_validate(
        estimator = data_pipe, 
        cv = 10, 
        X = X_train, 
        y = y_train, 
        return_train_score = True
    )
)
data_vfold_score

data_metrics = data_vfold_score.agg(["mean", "sem"])

param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 50, 1)
}

data_tune_pipe = make_pipeline(data_preprocessor, KNeighborsClassifier())

data_tune_grid_recall = GridSearchCV(
    estimator = data_tune_pipe,
    param_grid = param_grid, 
    cv = 10, 
    return_train_score = True,
    scoring = "recall",
    n_jobs = -1,
)

model_grid_recall = data_tune_grid_recall.fit(X_train, y_train)
accuracies_grid_recall = pd.DataFrame(model_grid_recall.cv_results_)

cv_plot_recall = alt.Chart(accuracies_grid_recall, title = "Graph 1 - Recall Estimate vs. k-value").mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("K"),
    y=alt.Y("mean_test_score").title("Recall Estimate").scale(zero=False)
)
cv_plot_recall

alt.Chart(...)

From the recall versus *k*-value graph generated, we see the highest peak to correlate with *k* = 3. Therefore, we can read from this that the highest validation recall occurs when *k* = 3. It is extrapolated from this graph that *k* = 3 yields the greatest recall. We thus run the model on our testing data with a *k*-value of 3, and we generate another confusion matrix.

In [33]:
knn1 = KNeighborsClassifier(n_neighbors = 3)

knn_pipeline1 = make_pipeline(data_preprocessor, knn1)
knn_pipeline1.fit(X_train, y_train)

data_test_predictions1 = data_test.assign(
    predict = knn_pipeline1.predict(data_test[["thal", "ca", "slope", "exang", "cp", "chol", "trestbps"]])
)

data_test_predictions1["predicted"] = data_test_predictions1["predict"] > 0

data_prediction_accuracy = knn_pipeline1.score(X_test, y_test)

data_mat1 = pd.crosstab(
    data_test_predictions1["heart_disease"], 
    data_test_predictions1["predicted"]
)
data_mat1

predicted,False,True
heart_disease,,
False,35,8
True,6,26


**Confusion Matrix 2 - *k* = 3**

We can see from the generated confusion matrix that the metrics are somewhat satisfactory. We find recall and run again but for accuracy. 

In [34]:
from sklearn.metrics import recall_score, precision_score
recall_score(
    y_true = data_test_predictions1["heart_disease"], 
    y_pred = data_test_predictions1["predicted"], 
    pos_label = True
)

0.8125

We see that the recall when *k* = 3 is 0.8125, which is a marked improvement from a recall of 0.656 when *k* = 2. We repeat this, but now for accuracy. 

In [35]:
np.random.seed(1234)

data_tune_grid_acc = GridSearchCV(
    estimator = data_tune_pipe,
    param_grid = param_grid, 
    cv = 10, 
    return_train_score = True,
    n_jobs = -1,
)
data_tune_grid_acc

model_grid_acc = data_tune_grid_acc.fit(X_train, y_train)
accuracies_grid_acc = pd.DataFrame(model_grid_acc.cv_results_)

cv_plot_acc = alt.Chart(accuracies_grid_acc, title = "Graph 2 - Accuracy Estimate vs. k-value").mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("K"),
    y=alt.Y("mean_test_score").title("Accuracy Estimate").scale(zero=False)
)
cv_plot_acc

alt.Chart(...)

In [27]:
data_prediction_accuracy = knn_pipeline1.score(X_test, y_test)
data_prediction_accuracy

0.8133333333333334

From the accuracy versus *k*-value graph above, we see that the peak accuracy value also occurs when *k* = 3. We calculate for accuracy using Confusion Matrix 2, as they use the same *k*-value. We thus find accuracy to be 0.813, fractionally higher when *k* = 2. 